In [1]:
import numpy as np
import pandas as pd

In [2]:
# Data Augmentation

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
train_gen = ImageDataGenerator(rescale=(1./255),horizontal_flip=True,shear_range=0.2)
test_gen = ImageDataGenerator(rescale=(1./255))  #--> (0 to 255) convert to (0 to 1)

In [4]:
train = train_gen.flow_from_directory('Dataset(original)/train',
                                      target_size=(224, 224),
                                      class_mode='categorical', 
                                      batch_size=8)
test = test_gen.flow_from_directory('Dataset(original)/test',
                                    target_size=(224, 224),
                                      class_mode='categorical', 
                                      batch_size=8)

Found 655 images belonging to 17 classes.
Found 183 images belonging to 17 classes.


In [5]:
train.class_indices

{'Darier_s disease': 0,
 'Muehrck-e_s lines': 1,
 'aloperia areata': 2,
 'beau_s lines': 3,
 'bluish nail': 4,
 'clubbing': 5,
 'eczema': 6,
 'half and half nailes (Lindsay_s nails)': 7,
 'koilonychia': 8,
 'leukonychia': 9,
 'onycholycis': 10,
 'pale nail': 11,
 'red lunula': 12,
 'splinter hemmorrage': 13,
 'terry_s nail': 14,
 'white nail': 15,
 'yellow nails': 16}

In [6]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121, VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping

In [7]:
# Set hyperparameters and configurations
num_classes = 17
batch_size = 32
learning_rate = 0.001
num_epochs = 30

In [8]:
# Define your DenseNet-VGG16 hybrid model architecture
def DenseNetVGG16(num_classes):
    # Load pre-trained DenseNet and VGG16 models
    densenet = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Freeze the pre-trained layers
    for layer in densenet.layers:
        layer.trainable = False
    for layer in vgg16.layers:
        layer.trainable = False

    # Combine DenseNet and VGG16 models
    input_layer = tf.keras.Input(shape=(224, 224, 3))
    densenet_output = densenet(input_layer)
    vgg16_output = vgg16(input_layer)

    # Add global average pooling layer
    densenet_output = GlobalAveragePooling2D()(densenet_output)
    vgg16_output = GlobalAveragePooling2D()(vgg16_output)

    # Concatenate DenseNet and VGG16 outputs
    combined_output = tf.keras.layers.concatenate([densenet_output, vgg16_output])

    # Add a fully connected layer
    dense = Dense(units=128, activation='relu')(combined_output)

    # Add output layer
    output = Dense(units=num_classes, activation='softmax')(dense)

    # Create the model
    model = Model(inputs=input_layer, outputs=output)
    
    return model

In [9]:
# Load and preprocess the nail image dataset
# Ensure that you have a data generator providing batches of image-label pairs
# Initialize your DenseNet-VGG16 hybrid model
model = DenseNetVGG16(num_classes)

In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 densenet121 (Functional)       (None, 7, 7, 1024)   7037504     ['input_3[0][0]']                
                                                                                                  
 vgg16 (Functional)             (None, 7, 7, 512)    14714688    ['input_3[0][0]']                
                                                                                                  
 global_average_pooling2d (Glob  (None, 1024)        0           ['densenet121[0][0]']        

In [11]:
# Create the optimizer with a specific learning rate
optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)

# Compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='accuracy', patience=3, restore_best_weights=True)

In [13]:
# Train the model
model.fit(train,epochs=num_epochs,validation_data=test, callbacks=[early_stopping])

Epoch 1/30
82/82 [==============================] - 183s 2s/step - loss: 2.7966 - accuracy: 0.1282 - val_loss: 2.3961 - val_accuracy: 0.2787
Epoch 2/30
82/82 [==============================] - 114s 1s/step - loss: 2.2773 - accuracy: 0.3084 - val_loss: 1.9984 - val_accuracy: 0.3989
Epoch 3/30
82/82 [==============================] - 102s 1s/step - loss: 1.9202 - accuracy: 0.3985 - val_loss: 1.6590 - val_accuracy: 0.5574
Epoch 4/30
82/82 [==============================] - 103s 1s/step - loss: 1.6496 - accuracy: 0.5176 - val_loss: 1.3833 - val_accuracy: 0.6066
Epoch 5/30
82/82 [==============================] - 104s 1s/step - loss: 1.4219 - accuracy: 0.5710 - val_loss: 1.0932 - val_accuracy: 0.7377
Epoch 6/30
82/82 [==============================] - 106s 1s/step - loss: 1.2452 - accuracy: 0.6565 - val_loss: 1.0620 - val_accuracy: 0.7158
Epoch 7/30
82/82 [==============================] - 106s 1s/step - loss: 1.0598 - accuracy: 0.6855 - val_loss: 0.8482 - val_accuracy: 0.8142
Epoch 8/30
82

In [14]:
# Save the trained model
model.save('DenseNetVGG16.h5')

In [15]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [16]:
model = load_model('DenseNetVGG16.h5')

In [24]:
img=image.load_img('Dataset-3/Test/Leukonychia/21.PNG',target_size=(224,224))

In [25]:
img = image.img_to_array(img)
img = np.expand_dims(img,axis=0)
model.predict(img)
pred = np.argmax(model.predict(img))
output = ['Darier_s disease','Muehrck-e_s lines','Onychogryphosis','Onycholycis_NailPsoriasis',
          'aloperia areata','beau_s lines','bluish nail', 'clubbing','eczema','half and half nailes (Lindsay_s nails)',
          'koilonychia','leukonychia','pale nail','red lunula',
          'splinter hemmorrage_Acral Lentiginous Melanoma',
          'terry_s nail_WhiteNails','yellow nails']
print(output[pred])

1/1 [==============================] - 0s 137ms/step
terry_s nail_WhiteNails
